## SKFlow Simple Linear Regression on Parkinson's Disease Speech Dataset

https://archive.ics.uci.edu/ml/datasets/Parkinson+Speech+Dataset+with++Multiple+Types+of+Sound+Recordings

In [2]:
import pandas as pd
import tensorflow as tf
import tensorflow.contrib.learn.python.learn as skflow
from sklearn import datasets, metrics
import numpy as np
import os.path
import pickle

In [30]:
### Converting Data into Tensors
def convert_dataframe(dataframe, colnames, labelcol):
    X = np.array(dataframe[colnames].iloc[:,]).astype(np.float32)
    Y = np.array(dataframe[labelcol].iloc[:,]).astype(np.float32)  
    return X, Y

### load or generate random ids for test and training sets
def slice_ids(length,run_id):
    idfile = "models/"+run_id+"_ids"
    if os.path.isfile(idfile): 
        ### Re-run model
        with open(idfile, 'rb') as fp:
            randomInd = pickle.load(fp)
    else:
        ### First run
        randomInd = np.random.permutation(length)
        with open(idfile, 'wb') as fp:
            pickle.dump(randomInd, fp)
    
    ###  70% of ids randomly selected for training
    ###  30% of ids randomly selected for testing
    mid = int(.7*length)
    trainidx = randomInd[:mid]
    testidx = randomInd[mid:]
    return trainidx, testidx

def run_LinearRegressor(run_id, steps):
    ### Get Training and Testing indexes
    trainidx, testidx = slice_ids(len(X),run_id)

    ### Define classifier - Simple Linear Regression
    ### model_dir - this is where the model is saved. To re-run use the same runIdentifier
    feature_columns = skflow.infer_real_valued_columns_from_input(X)
    model = skflow.LinearRegressor(
        feature_columns=feature_columns,
        model_dir="models/"+run_id,
        enable_centered_bias=False)
    
    model.fit(X[trainidx,:], 
              Y[trainidx],#.astype(np.int), 
              steps=steps)
    
    y_p = model.predict(X[testidx,:])
    y_t = Y[testidx]
    return np.linalg.norm(y_t-y_p)/y_t.shape[0]

def run_DNNClassifier(run_id, steps):
    ### Get Training and Testing indexes
    trainidx, testidx = slice_ids(len(X),run_id)

    ### Define classifier - DNN
    ### model_dir - this is where the model is saved. To re-run use the same runIdentifier
    model = skflow.DNNClassifier(
        hidden_units=[48, 24, 24], 
        n_classes= 3, # set to be the number of distinct categories of labels
        feature_columns=feature_columns,
        model_dir="models/"+run_id)
    
    model.fit(X[trainidx,:], 
              Y[trainidx].astype(np.int), 
              steps=steps)
    
    y_p = model.predict(X[testidx,:])
    y_t = Y[testidx]
    return eval_classifier(y_p,y_t)

### Load data and convert to tensors

In [6]:
COLUMN_NAMES = ["subject_id", "jitter_local", "jitter_local_absolute", "jitter_rap", "jitter_ppq5",
                 "jitter_ddp","shimmer_local","shimmer_local_db","shimmer_apq3","shimmer_apq5",
                 "shimmer_apq11","shimmer_dda","ac","nth","htn","pitch_median","pitch_mean","pitch_stddev",
                 "pitch_min","pitch_max","number_of_pulses", "number_of_periods", "period_mean",
                 "period_stddev","locally_unvoiced_frames_fraction","number_of_voice_breaks",
                 "degree_of_voice_breaks","updrs","class_information"]


datafile = "data/train_data.txt"
df_data = pd.read_csv(datafile, names=COLUMN_NAMES, header=None)

### test columns are the same but doesn't contain "UPDRS"
#COLUMN_NAMES.remove("updrs")
#df_test = pd.read_csv('data/test_data.txt', names = COLUMN_NAMES,header=None)

### Create list of column names for feature data.
featcol = list(df_data.columns.values)
featcol.remove("subject_id")
featcol.remove("class_information")
featcol.remove("updrs")

### First Model fitting UPDRS - SLR Classifier

In [36]:
labelcol = "updrs" 
X, Y = convert_dataframe(df_data, featcol, labelcol)
run_LinearRegressor("SLR_UPDRS_001", 1000)

Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


0.85746931418394434

### First Model fitting Class_Information - DNN Classifier

In [37]:
labelcol = "class_information"
X, Y = convert_dataframe(df_data, featcol, labelcol)
run_DNNClassifier("DNN_Class_001", 1000)

Explicitly set `enable_centered_bias` to 'True' if you want to keep existing behaviour.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.
Instructions for updating:
The default behavior of predict() is changing. The default value for
as_iterable will change to True, and then the flag will be removed
altogether. The behavior of this flag is described below.


0.650641025641
